In [1]:
%pip install flask flask-sqlalchemy flask-session flask-login torchvision
%pip install qwen-vl-utils transformers accelerate pdf2image bitsandbytes
%pip install faiss-cpu sentence-transformers numpy
%pip install --ignore-installed flask
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !apt update
# !apt install poppler-utils -y

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached flask-3.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
  Using cached MarkupSafe-3.0.2-cp310-cp310-win_amd64.whl.metadata (4.1 kB)
Using cached flask-3.1.0-py3-none-any.whl (102 kB)
Using cached blinker-1.9.0-py3-none-any.whl (8.5 kB)
Using cached click-8.1.8-py3-none-any.whl (98 kB)
Using cached itsdangerous-2.2.0-py3-none-any.whl (16 kB)
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
Using cached werkzeug-3.1.3-py3-none-any.whl (224 kB)
Using cached MarkupSafe-3.0.2-cp310-cp310-win_amd64.whl (15 kB)
Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)
Note: you may need to restart the


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://download.pytorch.org/whl/cu118



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
from accelerate import Accelerator
import json
import sqlite3
from datetime import datetime
# from llm_axe.agents import OnlineAgent
from pathlib import Path
import os
from flask import Flask, request, jsonify, send_file, abort
from werkzeug.utils import secure_filename
import torch

d:\SPECIAL\Documatic\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
accelerator = Accelerator()
device = accelerator.device
print(device)

cuda


In [4]:
# default: Load the model on the available device(s)
# min_pixels = 256 * 28 * 28
# max_pixels = 1280 * 28 * 28
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct", 
    # torch_dtype="float16", 
    device_map={"": device}, 
    load_in_8bit=True
)

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# model = Qwen2VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2-VL-7B-Instruct",
#     torch_dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto",
# )

# default processer
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")

# The default range for the number of visual tokens per image in the model is 4-16384. You can set min_pixels and max_pixels according to your needs, such as a token count range of 256-1280, to balance speed and memory usage.
# min_pixels = 256*28*28
# max_pixels = 1280*28*28
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46
Loading checkpoint shards: 100%|██████████| 2/2 [00:19<00:00,  9.52s/it]


In [5]:
class SQLiteFileSummaryStorage:
    """Store file summaries using SQLite"""
    
    def __init__(self, db_path="file_summaries.db"):
        self.db_path = db_path
        self._ensure_database_exists()
    
    def _ensure_database_exists(self):
        """Create database and table if they don't exist"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS file_summaries (
                id INTEGER PRIMARY KEY,
                device_id TEXT,
                filename TEXT,
                filepath TEXT,
                summary TEXT,
                ocr TEXT,
                timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        
        conn.commit()
        conn.close()
    
    def save_summary(self, device_id, filename, filepath, summary, ocr):
        """Save a file summary"""
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            
            cursor.execute('''
                INSERT OR REPLACE INTO file_summaries (device_id, filename, filepath, summary, ocr)
                VALUES (?, ?, ?, ?, ?)
            ''', (device_id, filename, filepath, summary, ocr))
            
            conn.commit()
            conn.close()
            return True
        except Exception as e:
            print(f"Error saving summary: {e}")
            return False
    
    def get_summary(self, filename):
        """Retrieve a file summary"""
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            
            cursor.execute('''
                SELECT summary FROM file_summaries
                WHERE filename = ?
            ''', (filename,))
            
            result = cursor.fetchone()
            conn.close()
            
            return result[0] if result else None
        except Exception as e:
            print(f"Error retrieving summary: {e}")
            return None
    
    def get_summary_with_device_id(self, device_id):
        """Retrieve a file summary"""
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            
            cursor.execute('''
                SELECT id, filename, filepath FROM file_summaries
                WHERE device_id = ?
            ''', (device_id,))
            
            result = cursor.fetchall()
            conn.close()
            
            return result if result else None
        except Exception as e:
            print(f"Error retrieving summary: {e}")
            return None
    
    def get_summary_with_id(self, id):
        """Retrieve a file summary"""
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            
            cursor.execute('''
                SELECT filename, filepath FROM file_summaries
                WHERE id = ?
            ''', (id,))
            
            result = cursor.fetchone()
            conn.close()
            
            return result if result else None
        except Exception as e:
            print(f"Error retrieving summary: {e}")
            return None
    
    def get_all_summaries(self):
        """Get all stored summaries"""
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            
            cursor.execute('''
                SELECT filename, filepath, summary, timestamp 
                FROM file_summaries
            ''')
            
            results = cursor.fetchall()
            conn.close()
            
            return {row[0]: {
                'filepath': row[1],
                'summary': row[2],
                'timestamp': row[3]
            } for row in results}
        except Exception as e:
            print(f"Error retrieving summaries: {e}")
            return {}

In [6]:
def is_correct_format(messages: list[dict]) -> bool:
    """
    Checks if the messages are already in the correct format.
    
    Correct format:
    [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "message text"
                }
            ]
        }
    ]
    
    Args:
        messages (list[dict]): List of message dictionaries to validate
        
    Returns:
        bool: True if format is correct, False otherwise
    """
    try:
        if not isinstance(messages, list):
            return False
            
        for message in messages:
            # Check if basic structure exists
            if not isinstance(message, dict):
                return False
                
            if 'role' not in message or 'content' not in message:
                return False
                
            # Check role
            if not isinstance(message['role'], str):
                return False
                
            # Check content structure
            content = message['content']
            if not isinstance(content, list):
                return False
                
            # Validate each content item
            for item in content:
                if not isinstance(item, dict):
                    return False
                    
                if 'type' not in item or 'text' not in item:
                    return False
                    
                if item['type'] != 'text':
                    return False
                    
                if not isinstance(item['text'], str):
                    return False
                    
        return True
        
    except Exception:
        return False

def convert_message_format(messages: list[dict]) -> list[dict]:
    """
    Converts messages from simple format to the structured format required by the model.
    
    Input format:
    [
        {
            "role": "user",
            "content": "message text"
        }
    ]
    
    Output format:
    [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "message text"
                }
            ]
        }
    ]
    
    Args:
        messages (list[dict]): List of message dictionaries in simple format
        
    Returns:
        list[dict]: List of message dictionaries in structured format
    """
    converted_messages = []
    
    for message in messages:
        # Handle case where content is already in the correct format
        if isinstance(message.get('content'), list):
            converted_messages.append(message)
            continue
            
        # Convert simple string content to structured format
        converted_message = {
            "role": message["role"],
            "content": [
                {
                    "type": "text",
                    "text": message["content"]
                }
            ]
        }
        converted_messages.append(converted_message)
    
    return converted_messages

In [7]:
class JARVIS():

    # Your ask function will always receive a list of prompts
    # The prompts are in open ai prompt format
    #  example: {"role": "system", "content": "You are a helpful assistant."}
    # If your model supports json format, use the format parameter to specify that to your model.
    def ask(self, messages:list, format:str="", temperature:float=0.8):
        """
        Args:
            prompts (list): A list of prompts to ask.
            format (str, optional): The format of the response. Use "json" for json. Defaults to "".
            temperature (float, optional): The temperature of the LLM. Defaults to 0.8.
        """
        correct_format = is_correct_format(messages)
        if not correct_format:
            messages = convert_message_format(messages)
        # Preparation for inference
        text = processor.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )
        image_inputs, video_inputs = process_vision_info(messages)
        inputs = processor(
            text=[text],
            images=image_inputs,
            videos=video_inputs,
            padding=True,
            return_tensors="pt",
        )
        inputs = inputs.to("cuda")
        
        torch.cuda.empty_cache()
        # Inference: Generation of the output
        generated_ids = model.generate(**inputs, 
                                       max_new_tokens=4096,           # Adjust based on your desired summary length
                                       temperature=0.2,          # Balances diversity and coherence
                                       top_p=0.9,                # Top-p sampling for additional control
                                       do_sample=True,           # Enables sampling for diversity
                                       repetition_penalty=1.1    # Avoids repetitive outputs
                                       )
        generated_ids_trimmed = [
            out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
        ]
        output_text = processor.batch_decode(
            generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
        )
        if not correct_format:
            return output_text[0]
        return output_text

In [8]:
import os, platform, subprocess

def add_poppler_to_path():
    poppler_dir = os.path.join(os.getcwd(), "poppler\\Library")
    print(poppler_dir)
    system = platform.system().lower()
    if system == "windows":
        poppler_bin_dir = os.path.join(poppler_dir, "bin") # Poppler DLLs usually in bin folder
        # if hasattr(os, "add_dll_directory"): # Python 3.8+
        #     os.add_dll_directory(poppler_bin_dir)
        #     print("Here")
        # else:
        os.environ["PATH"] = poppler_bin_dir + os.pathsep + os.environ["PATH"]
    elif system in ("linux", "darwin"):
        lib_path_var = "LD_LIBRARY_PATH" if system == "linux" else "DYLD_LIBRARY_PATH"
        poppler_lib_dir = os.path.join(poppler_dir, system, "lib")
        if lib_path_var in os.environ:
            os.environ[lib_path_var] = poppler_lib_dir + os.pathsep + os.environ[lib_path_var]
        else:
            os.environ[lib_path_var] = poppler_lib_dir
    else:
        raise OSError("Unsupported operating system")

# Call this function BEFORE importing any libraries that depend on Poppler
add_poppler_to_path()

d:\SPECIAL\Documatic\poppler\Library


In [9]:
from pdf2image import convert_from_path
from PIL import Image, ImageOps
import os

def optimize_images(filepath, output_folder, dpi=150, compression="webp"):
    """Convert PDF to optimized images for OCR."""
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    images = convert_from_path(filepath, dpi=dpi)
    
    for i, img in enumerate(images):
        # Convert to grayscale
        img = img.convert("L")
        
        # Trim whitespace
        img = ImageOps.expand(ImageOps.crop(img, border=10), border=5)  # Adjust for better trimming

        # Save with compression
        output_path = os.path.join(output_folder, f"page_{i+1}.{compression}")
        img.save(output_path, format=compression.upper(), quality=80 if compression == "webp" else None)

        print(f"Saved: {output_path}")

In [10]:
def getText():
    text = ""
    import os
    image_files = sorted([f for f in os.listdir("temporary") if f.endswith(('.png', '.webp', '.jpg'))])
    from easyocr import Reader

    # Initialize the Reader (specify the languages you need, e.g., English and Hindi)
    reader = Reader(['en'], gpu=True)  # Set gpu=True if you have a GPU

    for file in image_files:
        img_path = "temporary/" + file
        # Perform OCR
        results = reader.readtext(img_path)

        # Display results
        t=f"{file}:\n\n"
        for (bbox, tex, confidence) in results:
            t += tex + "\n"
        text += t + "\n"
    return text

In [11]:
from pdf2image import convert_from_path
db = SQLiteFileSummaryStorage()
jarvis = JARVIS()
def generate_summary(filepath, name, query):
    content = []
    img_out = "temporary"
    optimize_images(filepath, img_out)
    ocr = getText()
    image_files = sorted([f for f in os.listdir(img_out) if f.endswith(('.png', '.webp', '.jpg'))])
    i = 0
    batches = (len(image_files) // 5) + (1 if len(image_files) % 5 != 0 else 0)
    previous = ""
    while image_files:
        length = len(image_files)
        for j in range(0, min(5, length)):
            content.append({
                    "type":"image",
                    "image":"temporary/"+image_files.pop(0),
                    "resized_height": 280,
                    "resized_width": 420,
                })
   
        content.append({"type": "text", "text": f"""[INSTRUCTION]
You are a document intelligence assistant specializing in structured, detailed, and searchable summaries. You MUST always output in the exact format provided below, even if the document has little content.

[OUTPUT FORMAT]
1. **Core Summary:** [Brief summary of the document]
2. **Document Details:**
   - **Primary Purpose:** [Function of the document]
   - **Document Type:** [Type of document]
   - **Target Audience:** [Intended users]
3. **Extracted Details:** 
   - **Names:** [List of names]
   - **Numbers:** [List of numbers, dates, IDs]
   - **Addresses:** [If available]
   - **Keywords/Phrases:** [Relevant search terms]
4. **Alternate Use Cases:** 
   - [List at least 3 alternative ways this document could be useful]
5. **Searchable Contexts:** 
   - "This document is relevant when searching for..."
6. **Search Keywords:** 
   - [List 12-15 keywords]
7. **Alternate Search Phrases:**  
   - ["Example query 1", "Example query 2"]

[CONSTRAINTS]
- **Never return an incomplete output.**
- **If there is not enough information, reinstate the information using more generalised words, statemens and phrases and still fill all the sections**
- **Do not skip sections, always provide placeholders.** 
- **If a previous summary is provided, Improve on it with the new content found new next batch of images.**  


[User Input]
File: A set of images/PDF pages from a document (processed in batches).
User Description: {query}.
File Name: {name}.
Batch: {i+1}/{batches}
{"Previous summary: " + previous if i != 0 else ""}
"""})
#Batch Info: {batch}/{total} (if applicable).
#     content.append({"type": "text", "text": f"""You are a document summarization assistant. Your task is to create a comprehensive, searchable summary of a file based on its contents and provided description. Follow this structured approach:
# GUIDELINES:

# Focus on discoverability - include terms users might search for
# Incorporate both technical and layman's terms
# Add every keyword like any names, phrases etc.
# Consider different ways users might try to find this document
# Include crucial context from both the file content and user description
# Add industry-standard terminology where relevant
# Consider various use cases and access patterns
# Maintain factual accuracy while optimizing for searchability

# Now, analyze the provided file and description to generate a comprehensive summary like this, Do not exceed the numbers mentioned:

# INPUTS:
# File: attached pages as images. treat them combined as single file
# User Description: {query}
# File name: {name}
# OCR content: {ocr}
# Generate a summary in the following format:
# CORE SUMMARY (2-3 sentences):
# [Provide a clear, concise overview of the main content and purpose of the file]

# DOCUMENT TYPE & PURPOSE:
# - Primary Purpose: [Main function/goal of the document]
# - Document Category: [Report/Specification/Manual/Analysis/etc.]
# - Target Audience: [Intended readers/users]

# OCR HIGHLIGHTS:
# Summarize the OCR content and maintain all the important info

# KEY TOPICS:
# - [Topic 1]
# - [Topic 2]
# - [Topic 3]
# [List 3-5 main topics covered]

# REFERENCE CONTEXTS:
# "This file would be relevant when looking for..."
# - [Scenario 1]
# - [Scenario 2]
# - [Scenario 3]
# [List 3-4 specific use cases or scenarios]

# ADDITIONAL CONTEXT FROM USER DESCRIPTION:
# [Integrate any unique insights or context from the user's description that aren't apparent from the file content alone]

# SEARCH KEYWORDS:
# [List 8-12 relevant search terms, including:
# - Common synonyms
# - Technical terms
# - Related concepts
# - Alternative phrasings
# Separate with commas]

# IMPORTANT INFO:
# [List all the specific things in the document, including:
# - Names (places, people, emails, objects, anything)
# - Dates, Numbers
# - phrases
# Separate with commas]

# ALTERNATE SEARCH PHRASES:
# [List different phrases a user can use to look for this document, keep it very generalised, like any alternate use case except the primary one]

# """})
        print(content)
        messages = [
            {
                "role": "user",
                "content": content,
            }
        ]
        
        output_text = jarvis.ask(messages)
        previous = output_text
        content = []
        i += 1
    for page in image_files:
        if os.path.exists("temporary/"+page):
            os.remove("temporary/"+page)
    return previous, ocr

def get_file(query):
    jarvis = JARVIS()
    summaries = db.get_all_summaries()
    print(len(summaries.items()))
    print(query)
    scores = ""
    for filename, data in summaries.items():
        summary = data['summary']
        path = data['filepath']
        print("Summary: " + summary)
        print("Query: " + query)
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": f"""You are a document matching expert. Analyze how well a query matches a given summary and provide a single confidence score from 0-100.

Scoring Criteria:
1. Exact Keyword Matches (40 points max):
   - Exact matches of query terms (case-insensitive)
   - Multiple occurrences increase score
   - Technical/specific terms worth more than common words
   - Title/key position matches worth more

2. Keyword Variations (30 points max):
   - Plurals/singulars (car/cars)
   - Common synonyms (vehicle/automobile)
   - Word forms (run/running/ran)
   - Compound variations (healthcare/health care)

3. Semantic Relevance (20 points max):
   - Topic and concept alignment
   - Context relevance

4. Additional Factors (10 points max):
   - Term proximity
   - Information relevance

Query: {query}
Summary: {summary}

Follow these steps silently:
1. Check exact keyword matches
2. Find keyword variations
3. Assess semantic alignment
4. Consider additional factors
5. Sum up scores

Output only this format:
Confidence Score: <0-100>"""},
                ],
            }
        ]
        
        output_text = jarvis.ask(messages)
        scores += f"({filename},{path},{str(output_text[0])})\n"
    return scores

In [14]:
summary = generate_summary("d:/SPECIAL/Documatic/Flair Airlines.pdf", "Flair Airlines.pdf", "My airline ticket")
print(summary)
# db.save_summary("Indian License.pdf", "Indian License.pdf", summary[0])
# print(get_file("my indian driving license"))

Saved: temporary\page_1.webp
Saved: temporary\page_2.webp
[{'type': 'image', 'image': 'temporary/page_1.webp', 'resized_height': 280, 'resized_width': 420}, {'type': 'image', 'image': 'temporary/page_2.webp', 'resized_height': 280, 'resized_width': 420}, {'type': 'text', 'text': 'You are a document intelligence assistant specializing in **multi-page document processing**. Your task is to analyze **batches of document pages** and generate a **structured, detailed, and searchable summary** for retrieval using natural language queries.\n\n# TASK:\n1. **Process Document Pages in Batches**:  \n   - The document will be provided in multiple parts (batches of pages).  \n   - Ensure **consistency across batches** while summarizing.  \n   - Accumulate important details across batches to **build a comprehensive summary**.  \n\n2. **Preserve Key Details**:  \n   - Include all identifiable details (names, numbers, dates, keywords).  \n   - Ensure extracted details remain **consistent and linked** 

In [12]:
import faiss
import numpy as np
import json
from sentence_transformers import SentenceTransformer
from typing import List, Dict
import os

class DocumentSearchSystem:
    def __init__(self, index_file: str = "faiss_index.bin", metadata_file: str = "metadata.json"):
        # Initialize the Sentence-BERT model for embedding generation
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        
        # File paths for persistence
        self.index_file = index_file
        self.metadata_file = metadata_file
        self.database = SQLiteFileSummaryStorage()

        # List to store document metadata
        self.documents = []

        # Load or initialize FAISS index
        self.dimension = 384  # Embedding size of 'all-MiniLM-L6-v2'
        self.index = self._load_or_initialize_index()
        
        # # Load document metadata
        # self._load_metadata()
        
    def _load_or_initialize_index(self):
        """Load the FAISS index from disk or initialize a new one if not found."""
        if os.path.exists(self.index_file):
            index = faiss.read_index(self.index_file)
            print("FAISS index loaded from disk.")
        else:
            index = faiss.IndexFlatL2(self.dimension)  # L2 (Euclidean) index
            print("New FAISS index created.")
        return index

    # def _load_metadata(self):
    #     """Load document metadata from JSON file if it exists."""
    #     if os.path.exists(self.metadata_file):
    #         with open(self.metadata_file, "r") as f:
    #             self.documents = json.load(f)
    #         print("Document metadata loaded from disk.")
    #     else:
    #         self.documents = []
    
    def _save_index_and_metadata(self):
        """Save the FAISS index and document metadata to disk."""
        # Save the FAISS index
        faiss.write_index(self.index, self.index_file)
        
        # Save the metadata as JSON
        # with open(self.metadata_file, "w") as f:
        #     json.dump(self.documents, f)
        
        print("FAISS index and document metadata saved to disk.")

    def add_document(self, id: str, file_name: str, file_path: str, summary: str, ocr:str):
        """Add a document to the FAISS index and store its metadata."""
        # Generate the embedding for the summary
        final = "OCR:\n" + ocr + "\n\nSummary:\n" + summary
        embedding = self.model.encode(final).astype(np.float32).reshape(1, -1)
        
        # Add embedding to FAISS index
        self.index.add(embedding)
        
        # Store metadata in documents list
        # self.documents.append({
        #     "id": id,
        #     "file_name": file_name,
        #     "file_path": file_path,
        #     "summary": summary
        # })
        self.database.save_summary(device_id=id, filename=file_name, filepath=file_path, summary=summary, ocr=ocr)
        
        print(f"Added document: {file_name}")
        
        # Save index and metadata
        self._save_index_and_metadata()

    def search_documents(self, doc_id: str, query: str, top_k: int = 3) -> List[Dict]:
        """Search for documents matching the query and return top_k results."""
        # Generate embedding for the query
        query_embedding = self.model.encode(query).astype(np.float32).reshape(1, -1)
    
        documents = self.database.get_summary_with_device_id(doc_id)
        print("Docs: " + str(documents))

        # Get indices of documents with matching doc_id
        matching_indices = [doc[0] for i, doc in enumerate(documents)]
        print("Result: " + str(matching_indices))
        
        if not matching_indices:
            return []  # Return empty list if no documents match the ID
        
        # Create a subset index with only the matching documents
        subset_index = faiss.IndexFlatL2(self.dimension)
        subset_embeddings = np.vstack([
            self.index.reconstruct(i) for i in matching_indices
        ])
        subset_index.add(subset_embeddings)
        print("Subset: " + str(subset_embeddings.shape))
        
        # Perform search in subset index
        distances, indices = subset_index.search(query_embedding, min(top_k, len(matching_indices)))
        print("Indices: " + str(indices))
        
        # Retrieve and return matching documents
        results = []
        for idx, distance in zip(indices[0], distances[0]):
            if idx != -1:  # Ensure valid index
                # doc = self.documents[idx]
                # doc['score'] = 1 / (1 + distance)  # Convert distance to similarity score
                # results.append(doc)

                original_idx = matching_indices[idx]
                doc = list(self.database.get_summary_with_id(original_idx + 1))
                #doc = self.documents[original_idx].copy()  # Copy the metadata
                doc.append(float(1 / (1 + distance)))  # Convert to Python float
                results.append(doc)
        
        return results


In [32]:
system = DocumentSearchSystem()
system.add_document("Device 1", "File 1", "Path 1", "Summary 1", "OCR 1")
system.search_documents("Device 1", "Query 1")

FAISS index loaded from disk.
Added document: File 1
FAISS index and document metadata saved to disk.
Result: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


[['File 1', 'Path 1', 0.5000023245811462],
 ['File 1', 'Path 1', 0.44425609707832336],
 ['File 1', 'Path 1', 0.44425609707832336]]

In [ ]:
app = Flask(__name__)

# Define upload folder
UPLOAD_FOLDER = '/content/uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
search_system = DocumentSearchSystem()

@app.route('/upload', methods=['POST'])
def upload_file_and_text():
    # Get the text data from the request
    print("Here")
    text = request.form['text']
    id = request.form['id']
    
    if 'file' not in request.files:
        # scores = get_file(text)
        # print(scores)
        print("Search id: " + id + ", Text: " + text)
        results = search_system.search_documents(doc_id=id, query=text)
        print(results)
        output = {}
        i = 1
        for r in results:
            output[str(i)] = [r[0], r[1], str(r[2])]
            i += 1
        return jsonify({"text": output, "got_file": "true"}) 
    else:
        file = request.files['file']
        path = request.form['path']
        # Save the file securely
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
        file.save(file_path)
    
        summary, ocr = generate_summary(file_path, file.filename, text)
        #db.save_summary(file.filename, path, summary)
        search_system.add_document(
                id=id,
                file_name=file.filename,
                file_path=path,
                summary=summary,
                ocr=ocr
            )
        print("ID: " + id + "Saved: " + path + "," + summary)
    
        # Clean up: Delete the temporary file
        if os.path.exists(file_path):
            os.remove(file_path)
        
        # Return a response with the text and file path (for example purposes)
        output = {"text": "Saved: " + path, "file_saved": file_path}
        return jsonify(output)
        # if use_internet:
        #     output = online_jarvis.search(text)
        #     print("Internet: "+ output)
        #     return jsonify({"text": output})
    # print(output[0])
    # return jsonify({"text": output[0]})
            

@app.route('/download', methods=['GET'])
def download_file():
    # Get the filename from the query parameters
    filename = request.args.get('filename')
    
    # Construct the full path
    file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)

    # Check if the file exists
    if not os.path.isfile(file_path):
        abort(404)  # Return 404 if the file is not found

    # Send the file as an attachment
    return send_file(file_path, as_attachment=True)

app.run(host="0.0.0.0", port=5000)

FAISS index loaded from disk.
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.2.14:5000
Press CTRL+C to quit


Here
Search id: BE418B70-D927-EB11-80D9-089798CE56FE, Text: resume
Docs: [(1, 'Coop Work Permit.pdf', 'D:\\Google Drive Backup\\Personal Documents\\Coop Work Permit.pdf'), (2, 'Coop Work Permit.pdf', 'D:\\Google Drive Backup\\Personal Documents\\Coop Work Permit.pdf'), (3, 'Smart Serve.pdf', 'D:\\Google Drive Backup\\Personal Documents\\Smart Serve.pdf'), (4, 'OFHC_Food Handler Certificate.pdf', 'D:\\Google Drive Backup\\Personal Documents\\OFHC_Food Handler Certificate.pdf'), (5, 'Indian License.pdf', 'D:\\Google Drive Backup\\Indian License.pdf'), (6, 'Flair Airlines.pdf', 'D:\\SPECIAL\\Documatic\\Flair Airlines.pdf'), (7, 'Dishu_Bansal_Software_Engineer.pdf', 'D:\\Full time\\ElectricMind\\Dishu_Bansal_Software_Engineer.pdf')]
Result: [1, 2, 3, 4, 5, 6, 7]
Subset: (7, 384)


127.0.0.1 - - [30/Mar/2025 22:43:55] "POST /upload HTTP/1.1" 200 -


Indices: [[3 5 0]]
[['Indian License.pdf', 'D:\\Google Drive Backup\\Indian License.pdf', 0.37955552339553833], ['Dishu_Bansal_Software_Engineer.pdf', 'D:\\Full time\\ElectricMind\\Dishu_Bansal_Software_Engineer.pdf', 0.378853440284729], ['Coop Work Permit.pdf', 'D:\\Google Drive Backup\\Personal Documents\\Coop Work Permit.pdf', 0.3741218149662018]]
Here
Search id: BE418B70-D927-EB11-80D9-089798CE56FE, Text: my resume


127.0.0.1 - - [30/Mar/2025 22:44:32] "POST /upload HTTP/1.1" 200 -


Docs: [(1, 'Coop Work Permit.pdf', 'D:\\Google Drive Backup\\Personal Documents\\Coop Work Permit.pdf'), (2, 'Coop Work Permit.pdf', 'D:\\Google Drive Backup\\Personal Documents\\Coop Work Permit.pdf'), (3, 'Smart Serve.pdf', 'D:\\Google Drive Backup\\Personal Documents\\Smart Serve.pdf'), (4, 'OFHC_Food Handler Certificate.pdf', 'D:\\Google Drive Backup\\Personal Documents\\OFHC_Food Handler Certificate.pdf'), (5, 'Indian License.pdf', 'D:\\Google Drive Backup\\Indian License.pdf'), (6, 'Flair Airlines.pdf', 'D:\\SPECIAL\\Documatic\\Flair Airlines.pdf'), (7, 'Dishu_Bansal_Software_Engineer.pdf', 'D:\\Full time\\ElectricMind\\Dishu_Bansal_Software_Engineer.pdf')]
Result: [1, 2, 3, 4, 5, 6, 7]
Subset: (7, 384)
Indices: [[5 3 0]]
[['Dishu_Bansal_Software_Engineer.pdf', 'D:\\Full time\\ElectricMind\\Dishu_Bansal_Software_Engineer.pdf', 0.39481064677238464], ['Indian License.pdf', 'D:\\Google Drive Backup\\Indian License.pdf', 0.38389843702316284], ['Coop Work Permit.pdf', 'D:\\Google Driv